# Helpers

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import griddata

In [3]:
csv_dir = "../output/"

In [4]:
def parse_boundary(csv_path):
    data = pd.read_csv(csv_path, header=None)
    x, y, z = data[0], data[1], data[2]
    x, y = (x + 1) / 2, (y + 1) / 2
    return x, y, z

In [5]:
def visualize_csv_output(path, output_path="./solver_output_plot.png", boundary_path=None, xstart=0, ystart=0, xend=1, yend=1, surface=False):
    data = pd.read_csv(path, header=None)
    x = np.linspace(xstart, xend, data.shape[1])
    y = np.linspace(ystart, yend, data.shape[0])
    X, Y = np.meshgrid(x, y)
    Z = data.values         

    # Create a mask for valid points
    valid_mask = ~np.isnan(Z)

    # Filter X, Y, Z using the mask
    X_valid = X[valid_mask]
    Y_valid = Y[valid_mask]
    Z_valid = Z[valid_mask]

    # Interpolating to fill in NaN values for surface plotting
    Z_interp = griddata((X_valid, Y_valid), Z_valid, (X, Y), method='linear')

    # Create a scatter plot for valid points
    fig = go.Figure()

    # Add surface plot with interpolated Z values
    if surface: 
        fig.add_trace(go.Surface(z=Z_interp, x=X, y=Y, colorscale='Viridis', showscale=True))

    # Add a scatter plot for the valid points
    if not surface:
        fig.add_trace(go.Scatter3d(x=X_valid, y=Y_valid, z=Z_valid, mode='markers',
            marker=dict(size=2, color='blue')  # Customize marker appearance if needed
        ))
    
    # add boundary if provided
    if boundary_path is not None: 
        x_boundary, y_boundary, z_boundary = parse_boundary(boundary_path)
        fig.add_trace(go.Scatter3d(x=x_boundary, y=y_boundary, z=z_boundary, mode='markers',
            marker=dict(size=2, color='red')  
        ))

    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',

        # Set aspect ratio to ensure fixed scale
        aspectmode='cube'  # Ensures that all axes are scaled equally
    ))

    fig.show()
    fig.write_image(output_path)
    print(f"Figure saved as {output_path}")

# Example usage
# visualize_csv_output("your_file.csv")


In [6]:
import plotly.graph_objects as go
def print_boundary_dirichlet(path=None, shape=None, dir=None): 
    if path is None: path = dir + shape + 'BoundaryDirichlet.csv'
    x, y, z = parse_boundary(path)
    fig = go.Figure()
    for i in range(len(x)):
        fig.add_trace(go.Scatter3d(
            x=[x[i], x[(i + 1) % len(x)]],
            y=[y[i], y[(i + 1) % len(y)]],
            z=[z[i], z[(i + 1) % len(z)]],
            mode='lines',
            line=dict(color='blue', width=2)
        ))
    fig.add_trace(go.Scatter3d(x=x,y=y,z=z,mode='markers',
        marker=dict(size=5, color='red', opacity=0.8)  # Color points
    ))

    fig.update_layout(title_text='3D Star Shape on a Tilted Plane',
                  scene=dict(
                      xaxis_title='X-axis',
                      yaxis_title='Y-axis',
                      zaxis_title='Z-axis'),
                  )

    fig.show()
    

In [7]:
def get_shape_csv(shape): 
    return shape + ".csv"
def get_boundaryD_csv(shape): 
    return shape + "BoundaryDirichlet.csv"

# Plot Saddle

In [103]:
shape = "saddlePoint"
shape_csv = get_shape_csv(shape)
boundary_csv = get_boundaryD_csv(shape)

In [91]:
print_boundary_dirichlet(path=csv_dir + boundary_csv) 

In [93]:
visualize_csv_output(csv_dir + shape_csv, surface=False, boundary_path=csv_dir + boundary_csv)

Figure saved as ./solver_output_plot.png


# Plot Star

In [105]:
shape = "star"
shape_csv = get_shape_csv(shape)
boundary_csv = get_boundaryD_csv(shape)

In [107]:
print_boundary_dirichlet(path=csv_dir + boundary_csv) 

In [108]:
visualize_csv_output(csv_dir + shape_csv, surface=False, boundary_path=csv_dir + boundary_csv)

Figure saved as ./solver_output_plot.png


### Plot random height star

In [20]:
shape = "random_height_star"
shape_csv = get_shape_csv(shape)
boundary_csv = get_boundaryD_csv(shape)

In [26]:
print_boundary_dirichlet(path=csv_dir + boundary_csv) 

In [27]:
visualize_csv_output(csv_dir + "random_height_star.csv", surface=False, boundary_path=csv_dir + boundary_csv)

Figure saved as ./solver_output_plot.png


### Plot Cicle

In [28]:
shape = "circle"
shape_csv = get_shape_csv(shape)
boundary_csv = get_boundaryD_csv(shape)

In [29]:
print_boundary_dirichlet(path=csv_dir + boundary_csv) 

In [30]:
visualize_csv_output(csv_dir + shape_csv, surface=False, boundary_path=csv_dir + boundary_csv)

Figure saved as ./solver_output_plot.png


In [31]:
visualize_csv_output(csv_dir + shape_csv, surface=True, boundary_path=csv_dir + boundary_csv)

Figure saved as ./solver_output_plot.png


In [33]:
print_boundary_dirichlet(path=csv_dir + boundary_csv) 

In [32]:
visualize_csv_output(csv_dir + shape_csv, surface=False, boundary_path=csv_dir + boundary_csv)

Figure saved as ./solver_output_plot.png


In [35]:
shape = "circle3"
shape_csv = get_shape_csv(shape)
boundary_csv = get_boundaryD_csv(shape)

In [36]:
print_boundary_dirichlet(path=csv_dir + boundary_csv) 

In [37]:
visualize_csv_output(csv_dir + shape_csv, surface=False, boundary_path=csv_dir + boundary_csv)

Figure saved as ./solver_output_plot.png


### Constant Height Bubble 

In [44]:
shape = "bubble-constant"
shape_csv = get_shape_csv(shape)
boundary_csv = get_boundaryD_csv(shape)

In [45]:
print_boundary_dirichlet(path=csv_dir + boundary_csv) 

In [46]:
visualize_csv_output(csv_dir + shape_csv, surface=False, boundary_path=csv_dir + boundary_csv)

Figure saved as ./solver_output_plot.png


In [49]:
import numpy as np
import plotly.graph_objects as go

# Parameters for the soap bubble
num_points = 20 # Density of points in radial and angular directions
radius = 1.0      # Radius of the 2D bubble
height_amplitude = 0.05  # Amplitude of the height field perturbation

# Create polar coordinates for the 2D disk
r = np.linspace(0, radius, num_points)
theta = np.linspace(0, 2 * np.pi, num_points)
r, theta = np.meshgrid(r, theta)

# Convert polar coordinates to Cartesian coordinates (X, Y)
x = r * np.cos(theta)
y = r * np.sin(theta)

# Define height (Z) field with a minimal surface perturbation
z = height_amplitude * np.sin(5 * r) * np.cos(5 * theta)

# Define a color map for the iridescent effect
colorscale = [
    [0, 'rgb(255, 182, 193)'], 
    [0.5, 'rgb(173, 216, 230)'], 
    [1, 'rgb(255, 255, 255)']
]

# Create Plotly surface plot
fig = go.Figure(data=[go.Surface(x=x, y=y, z=z, surfacecolor=z, colorscale=colorscale)])

# Adjust the layout
fig.update_layout(
    title="2D Soap Bubble with Height Fields (3D Visualization)",
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectratio=dict(x=1, y=1, z=0.3)
    )
)

# Show the plot
fig.show()
